# 02 - EDA
___

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
import time

from nltk.corpus import stopwords
from datetime import datetime
from sklearn.manifold import TSNE

pd.options.display.max_columns = 35

## Word2Vec

In [2]:
# read in our card files
df = pd.read_csv('../Data/cards_cleaned.csv')
df.head()

,name,layout,colors,color_identity,mana_cost,cmc,type_line,card_type,super_type,sub_type,oracle_text,legalities,rarity,edhrec_rank,power,toughness,loyalty,card_faces,oracle_text_back,colors_back,power_back,toughness_back,loyalty_back,card_type_back,mana_cost_back
0,Static Orb,normal,[],[],{3},3.0,Artifact,Artifact,NONE,NONE,as long as static orb is untapped players can'...,legacy vintage commander duel,rare,1660.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
1,Sensory Deprivation,normal,['U'],['U'],{U},1.0,Enchantment — Aura,Enchantment,NONE,Aura,enchant creatureenchanted creature gets -3/-0,pioneer modern legacy pauper vintage penny com...,common,17172.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
2,Road of Return,normal,['G'],['G'],{G}{G},2.0,Sorcery,Sorcery,NONE,NONE,choose one return target permanent card from y...,legacy vintage commander duel,rare,5184.0,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
3,Storm Crow,normal,['U'],['U'],{1}{U},2.0,Creature — Bird,Creature,NONE,Bird,flying this creature can't be blocked except b...,modern legacy pauper vintage penny commander duel,common,10016.0,1,2,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE
4,Walking Sponge,normal,['U'],['U'],{1}{U},2.0,Creature — Sponge,Creature,NONE,Sponge,{t}: target creature loses your choice of flyi...,legacy vintage commander duel,uncommon,15762.0,1,1,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE


In [3]:
df.isnull().sum()

name                 0
layout               0
colors               0
color_identity       0
mana_cost            0
cmc                  0
type_line            0
card_type            0
super_type           0
sub_type             0
oracle_text          0
legalities          16
rarity               0
edhrec_rank          0
power                0
toughness            0
loyalty              0
card_faces           0
oracle_text_back     0
colors_back          0
power_back           0
toughness_back       0
loyalty_back         0
card_type_back       0
mana_cost_back       0
dtype: int64

In [4]:
# Let's make a list of all the words in our oracle_text
text_list = [text for text in df.loc[df['oracle_text'] != 'NONE']['oracle_text']]
text_list += [text for text in df.loc[df['oracle_text_back'] != 'NONE']['oracle_text_back']]

all_oracle_text = ' '.join(text_list)

len(all_oracle_text)

2517938

In [5]:
import gensim.downloader as api

In [6]:
corpus = api.load('text8')

In [7]:
iter(corpus)

<generator object Dataset.__iter__ at 0x1a234f9750>

In [8]:
all_oracle_text.split()[:10]

['as',
 'long',
 'as',
 'static',
 'orb',
 'is',
 'untapped',
 'players',
 "can't",
 'untap']

In [9]:
print(next(iter(corpus))[:100])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to', 'describe', 'any', 'act', 'that', 'used', 'violent', 'means', 'to', 'destroy', 'the', 'organization', 'of', 'society', 'it', 'has', 'also', 'been', 'taken', 'up', 'as', 'a', 'positive', 'label', 'by', 'self', 'defined', 'anarchists', 'the', 'word', 'anarchism', 'is', 'derived', 'from', 'the', 'greek', 'without', 'archons', 'ruler', 'chief', 'king', 'anarchism', 'as', 'a', 'political', 'philosophy', 'is', 'the', 'belief', 'that', 'rulers', 'are', 'unnecessary', 'and', 'should', 'be', 'abolished', 'although', 'there', 'are', 'differing']


In [10]:
from gensim.models.word2vec import Word2Vec
Word2Vec()

In [11]:
# Import Word2Vec
from gensim.models.word2vec import Word2Vec

# # If you want to use gensim's data, import their downloader
# # and load it.
# import gensim.downloader as api
# corpus = api.load('text8')

# If you have your own iterable corpus of cleaned data, you can 
# read it in as corpus and pass that in.

# Train a model! 
model = Word2Vec([all_oracle_text.split()],      # Corpus of data.
                 size=100,    # How many dimensions do you want in your word vector?
                 window=5,    # How many "context words" do you want?
                 min_count=1, # Ignores words below this threshold.
                 sg=0,        # SG = 1 uses SkipGram, SG = 0 uses CBOW (default).
                 workers=4)   # Number of "worker threads" to use (parallelizes process).

In [12]:
# Do what you'd like to do with your dataa!
model.most_similar("target")

/Users/Creighton/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('when', 0.9999192953109741),
 ('or', 0.9999061822891235),
 ('creature', 0.9999059438705444),
 ('you', 0.9999004602432251),
 ('and', 0.9998999238014221),
 ('create', 0.9998994469642639),
 ('an', 0.9998980760574341),
 ('{t}:', 0.9998965263366699),
 ('the', 0.99989253282547),
 ('a', 0.9998922944068909)]

In [13]:
model['target']

/Users/Creighton/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-7.64291167e-01,  1.09619536e-01,  1.31526485e-01,  7.75637865e-01,
       -2.94463277e-01,  4.98481020e-02,  7.99924791e-01, -6.29162043e-02,
       -3.09405148e-01,  6.13556683e-01, -3.03734481e-01,  1.16898008e-01,
        1.27129316e-01, -1.94684356e-01,  1.74577668e-01,  2.16631830e-01,
       -1.20956205e-01,  1.83368117e-01,  1.76627249e-01,  6.67207420e-01,
        1.86459929e-01, -1.70099333e-01,  6.07492700e-02,  2.15013530e-02,
        8.07961673e-02,  1.19495280e-01, -8.75089243e-02,  2.56813228e-01,
        6.47187591e-01,  2.01004803e-01,  3.15903872e-01,  1.59206241e-02,
       -2.39657966e-04,  2.81567037e-01, -3.04256707e-01,  1.59774482e-01,
        2.86358029e-01,  2.97388911e-01, -3.36614579e-01,  3.75639379e-01,
       -1.89142108e-01, -2.82462806e-01,  7.26493895e-02,  2.72799969e-01,
        1.04061596e-01,  5.53077638e-01, -3.50182384e-01, -1.65349439e-01,
        1.83457330e-01, -1.94742337e-01,  5.21933697e-02, -8.84974655e-03,
       -1.45818889e-01,  

In [14]:
model.vocabulary

In [15]:
# now let's get a list of the vectors for each word in our oracle text
word_vec = [model[word] for word in all_oracle_text.split()]

/Users/Creighton/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [16]:
# now let's make a dataframe for our oracle text words and their corresponding vectors
w2vdf = pd.DataFrame.from_dict(dict(zip(all_oracle_text.split(), word_vec))).T
w2vdf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,...,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
as,-0.712666,0.105894,0.118074,0.716828,-0.276016,0.051537,0.733176,-0.058576,-0.286623,0.578248,-0.283437,0.109298,0.124019,-0.182645,0.151869,0.205625,-0.117071,...,-0.249985,-0.031635,-0.381112,-0.362901,0.017250,0.353409,0.022426,0.637273,-0.546158,0.752871,0.185273,0.101595,0.106143,-0.557166,-0.220504,-0.412907,0.172609
long,-0.543973,0.072384,0.088101,0.550097,-0.200218,0.037154,0.564150,-0.046289,-0.219879,0.439153,-0.206775,0.082577,0.093084,-0.139129,0.123793,0.156807,-0.085480,...,-0.191412,-0.028843,-0.290480,-0.274328,0.008648,0.273312,0.014000,0.487628,-0.412857,0.575574,0.143477,0.084284,0.072474,-0.428785,-0.165880,-0.322704,0.128638
static,-0.026204,0.003002,0.000864,0.021676,-0.007224,0.005521,0.028247,-0.002364,-0.007170,0.022124,-0.006481,0.007626,0.005474,-0.010011,0.004657,0.008159,-0.007357,...,-0.013565,-0.003836,-0.011565,-0.008142,0.001608,0.009687,-0.000084,0.022031,-0.022348,0.027601,0.009174,0.008763,0.006268,-0.015685,-0.011126,-0.015540,0.009187
orb,-0.017982,0.007328,0.005021,0.024109,-0.006651,0.001299,0.023010,0.001417,-0.010873,0.016292,-0.012773,0.007572,-0.000884,-0.001782,0.003472,0.001712,0.000420,...,-0.007577,-0.003470,-0.014577,-0.012628,-0.002770,0.015727,-0.003447,0.016725,-0.019196,0.021136,0.008865,0.002437,-0.001518,-0.018682,-0.005439,-0.017565,0.007386
is,-0.729166,0.109495,0.126977,0.736984,-0.281400,0.047449,0.760224,-0.064757,-0.289541,0.589999,-0.288252,0.106949,0.124344,-0.185384,0.161635,0.202190,-0.112565,...,-0.246772,-0.040629,-0.383683,-0.372151,0.007313,0.357307,0.025711,0.645717,-0.558204,0.768484,0.187779,0.110246,0.106621,-0.565040,-0.224004,-0.426406,0.181875


In [17]:
w2vdf.shape

(11485, 100)

In [ ]:
t0 = time.time()

# Initialize t-SNE
tsne = TSNE(random_state = 42, perplexity = 400, n_iter=1000)

tsne_df = tsne.fit_transform(w2vdf)

print((time.time() - t0)/60)

In [ ]:
sns.set()
# Initialize figure
fig, ax = plt.subplots(figsize = (15, 15))
sns.scatterplot(tsne_df[:, 0], tsne_df[:, 1], alpha = 0.5)

# initialize list of texts
texts = []
words_to_plot = list(np.arange(1, 11485, 250))

# Append words to list
for word in words_to_plot:
    texts.append(plt.text(tsne_df[word, 0], tsne_df[word, 1], w2vdf.index[word], fontsize = 14))
    